# Chapter 4: Exercise 14

We develop a model to predict whether a given car gets high or low gas mileage.

**Import packages**

In [1]:
# import numpy as np
# import pandas as pd
# from matplotlib.pyplot import subplots
# import statsmodels.api as sm
from ISLP import load_data
# from ISLP.models import (ModelSpec as MS,
#                          summarize)

In [2]:
Auto = load_data('Auto')
Auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
